# GWAS 1: Window Size Analysis - SD = 2.0
Author: Sophie Sigfstead 

Purpose: This is a copy of the gwas_1_window_size_analysis_sd=1.0.ipynb, now with SD = 2.0. 

This is an in-depth analysis of loci size and how it affects the results of our method for GWAS 1 (Depression).
Specifically, my goal was to test various window sizes and see how this affected the accuracy of both our method (i.e., using the brain track set) versus a random set.

When I refer to window size, this is the distance around a significant snp that is eliminated in the GWAS 1 procedure. SNPs are selected by order of p-value, with any snps with base pair locations within the window size (to the left or right) being eliminated (on the basis that there would be LD etc.). This repeats until there are no SNPs remaining that meet the p-value threshold. In Dr. Cai's original study, she used +/- 1Mb as the window size, and we have replicated this procedure. However, this creates very large windows and potentially doesn't differentiate our method as well (to be seen in this notebook).

In the gwas_1_single_track_analysis directory, I performed multiple analyses comparing the results of our method using brain track sets versus random sets, using a 1Mb window size. Suprisingly, while the brain tracks were a bit better, they weren't as improved as we'd expect. On closer inspection, within the matching loci, the distance between our identified SNP and the study's identified SNP was at most 500kB, while the same was not necessarily true for the random tracks. As such, it may be that the 1Mb window is too large to differentiate our method - or, it could be that tissue specificity is not as important as we assumed.

In this notebook, I will provide the results of running our method with various window sizes (250kb, 375kb, 500kb, 750kb, 1Mb), comparing the brain track set ("reference") to the aggregated result of 16 random sets (id = a through p). The random sets contain tracks that are not brain tracks, and were sampled without replacement, so that the sets contain an diverse representation of all the tracks. 

In [1]:
import pandas as pd

window_sizes = [250000, 375000, 500000, 750000, 1000000]

keys = ['a', 'b', 'c', 'd', 'e', 'f','g','h','i','j','k','l','m','n','o','p']

coding_snps_file =  pd.read_csv('../gwas_1_single_track_analysis/coding_region_set.csv')
coding_snps_set = set(coding_snps_file['snp'])

In [2]:
reference_results = pd.DataFrame(columns = ['window', 'total_snps', 'total_coding', 'total_non_coding'])
# Create a reference table
for window in window_sizes: 
    reference_directory = f"snp_lists_results/id=reference/window={window}/filtered_snps_gwas_1_sd=0.0.csv"
    reference_snps = pd.read_csv(reference_directory)
    reference_coding_snps= len(reference_snps[reference_snps['snp'].isin(coding_snps_set)])
    reference_results = reference_results._append({'window': window, 'total_snps': len(reference_snps), 
                        'total_coding':reference_coding_snps, 'total_non_coding': (len(reference_snps)-reference_coding_snps)}, ignore_index=True)
    
reference_results.sort_values(by=['window'], ascending = False)

,window,total_snps,total_coding,total_non_coding
4,1000000,29,0,29
3,750000,31,0,31
2,500000,37,1,36
1,375000,37,0,37
0,250000,44,0,44


The above table simply outlines the reference set sizes for each window size. For example, for window size 250000, the reference set contains 44 total snps, 0 of which are coding.
Below, I will create a table for each window size. Note that all of the results are done using a 2.0 SD threshold in this notebook.

In [3]:
import pandas as pd

# Function to create a table for each window size
def create_window_size_results(window):
    # Initialize the main DataFrame
    df = pd.DataFrame(columns=["set", "total_snps_found", "total_snps_overlap", "total_loci_overlap", "total_coding_snps", "average_p_value"])

    # Create the reference result
    reference_data = pd.read_csv(f"combined_results/id=reference/window={window}/combined_result_sd=2.0.csv").iloc[0]
    reference_dict = {
        "set": "reference",
        "total_snps_found": reference_data['num_snps_found'],
        "total_snps_overlap": reference_data['num_snps_overlap'],
        "total_loci_overlap": reference_data['num_loci_overlap'],
        "total_coding_snps": reference_data['num_coding_snps'],
        "average_p_value": reference_data['p_value']
    }
    df = pd.concat([df, pd.DataFrame([reference_dict])], ignore_index=True)

    # Initialize a DataFrame to collect all random set data
    id_data = pd.DataFrame(columns=["set", "total_snps_found", "total_snps_overlap", "total_loci_overlap", "total_coding_snps", "average_p_value"])
    keys = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p']

    # Collect data for each random set (a - p)
    for key in keys:
        id_data_path = f"combined_results/id={key}/window={window}/combined_result_sd=2.0.csv"
        id_data_df = pd.read_csv(id_data_path).iloc[0]
        id_data_dict = {
            "set": key,
            "total_snps_found": id_data_df['num_snps_found'],
            "total_snps_overlap": id_data_df['num_snps_overlap'],
            "total_loci_overlap": id_data_df['num_loci_overlap'],
            "total_coding_snps": id_data_df['num_coding_snps'],
            "average_p_value": id_data_df['p_value']
        }
        id_data = pd.concat([id_data, pd.DataFrame([id_data_dict])], ignore_index=True)

    # Aggregate the statistics across all random sets
    agg_id_data_mean = {
        "set": "random_aggregated_mean",
        "total_snps_found": id_data['total_snps_found'].mean(),
        "total_snps_overlap": id_data['total_snps_overlap'].mean(),
        "total_loci_overlap": id_data['total_loci_overlap'].mean(),
        "total_coding_snps": id_data['total_coding_snps'].mean(),
        "average_p_value": id_data['average_p_value'].mean()
    }
    agg_id_data_median = {
        "set": "random_aggregated_median",
        "total_snps_found": id_data['total_snps_found'].median(),
        "total_snps_overlap": id_data['total_snps_overlap'].median(),
        "total_loci_overlap": id_data['total_loci_overlap'].median(),
        "total_coding_snps": id_data['total_coding_snps'].median(),
        "average_p_value": id_data['average_p_value'].median()
    }
    agg_id_data_max = {
        "set": "random_aggregated_max",
        "total_snps_found": id_data['total_snps_found'].max(),
        "total_snps_overlap": id_data['total_snps_overlap'].max(),
        "total_loci_overlap": id_data['total_loci_overlap'].max(),
        "total_coding_snps": id_data['total_coding_snps'].max(),
        "average_p_value": id_data['average_p_value'].max()
    }
    agg_id_data_min = {
        "set": "random_aggregated_min",
        "total_snps_found": id_data['total_snps_found'].min().astype(int),
        "total_snps_overlap": id_data['total_snps_overlap'].min(),
        "total_loci_overlap": id_data['total_loci_overlap'].min(),
        "total_coding_snps": id_data['total_coding_snps'].min(),
        "average_p_value": id_data['average_p_value'].min()
    }

    # Add the aggregated results to the main DataFrame
    df = pd.concat([df, pd.DataFrame([agg_id_data_mean])], ignore_index=True)
    df = pd.concat([df, pd.DataFrame([agg_id_data_median])], ignore_index=True)
    df = pd.concat([df, pd.DataFrame([agg_id_data_max])], ignore_index=True)
    df = pd.concat([df, pd.DataFrame([agg_id_data_min])], ignore_index=True)

    return df
   


### Result for window size = 1Mb

In [4]:
df = create_window_size_results(1000000)
df

/var/folders/hf/9rstklcn237dv4bv1x1d6x740000gn/T/ipykernel_20897/3988943723.py:18: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, pd.DataFrame([reference_dict])], ignore_index=True)
/var/folders/hf/9rstklcn237dv4bv1x1d6x740000gn/T/ipykernel_20897/3988943723.py:36: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  id_data = pd.concat([id_data, pd.DataFrame([id_data_dict])], ignore_index=True)


,set,total_snps_found,total_snps_overlap,total_loci_overlap,total_coding_snps,average_p_value
0,reference,45.0,6.0000,26.00,5.0000,3.616732e-07
1,random_aggregated_mean,42.0,6.9375,27.25,4.5625,3.082382e-07
2,random_aggregated_median,42.0,7.0000,27.00,4.5000,3.076975e-07
3,random_aggregated_max,45.0,8.0000,29.00,6.0000,3.627492e-07
4,random_aggregated_min,39.0,5.0000,25.00,3.0000,2.711803e-07


The total possible snps / overlap loci was 29. The performance of the brain tracks is worse than the random aggregated result.   In addition, we see that the number of matching snps in the random sets, on average is better than the reference set. We also see a lower number of snps found in the random tracks. 

### Result for window size = 750000

In [5]:
df = create_window_size_results(750000)
df

/var/folders/hf/9rstklcn237dv4bv1x1d6x740000gn/T/ipykernel_20897/3988943723.py:18: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, pd.DataFrame([reference_dict])], ignore_index=True)
/var/folders/hf/9rstklcn237dv4bv1x1d6x740000gn/T/ipykernel_20897/3988943723.py:36: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  id_data = pd.concat([id_data, pd.DataFrame([id_data_dict])], ignore_index=True)


,set,total_snps_found,total_snps_overlap,total_loci_overlap,total_coding_snps,average_p_value
0,reference,48.0000,6.0000,28.00,5.0000,3.616732e-07
1,random_aggregated_mean,44.8125,6.8125,29.25,4.5625,3.082382e-07
2,random_aggregated_median,45.0000,7.0000,29.00,4.5000,3.076975e-07
3,random_aggregated_max,48.0000,8.0000,31.00,6.0000,3.627492e-07
4,random_aggregated_min,41.0000,5.0000,27.00,3.0000,2.711803e-07


 The reference value here is 31 and we have the brain tracks recovering 28 snps and the random sets recovering 29.25. The number of matching snps is higher on average in the random set, which is again suprising. 

### Result for window size = 500000

In [6]:
df = create_window_size_results(500000)
df

/var/folders/hf/9rstklcn237dv4bv1x1d6x740000gn/T/ipykernel_20897/3988943723.py:18: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, pd.DataFrame([reference_dict])], ignore_index=True)
/var/folders/hf/9rstklcn237dv4bv1x1d6x740000gn/T/ipykernel_20897/3988943723.py:36: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  id_data = pd.concat([id_data, pd.DataFrame([id_data_dict])], ignore_index=True)


,set,total_snps_found,total_snps_overlap,total_loci_overlap,total_coding_snps,average_p_value
0,reference,54.0,6.00,34.00,7.00,3.616732e-07
1,random_aggregated_mean,48.5,7.25,35.25,6.25,3.082382e-07
2,random_aggregated_median,48.5,7.00,35.00,7.00,3.076975e-07
3,random_aggregated_max,51.0,9.00,37.00,8.00,3.627492e-07
4,random_aggregated_min,45.0,5.00,33.00,3.00,2.711803e-07


Similar results to above, the reference value is n = 37 with 1 coding snp. 
### Result for window size = 375000

In [7]:
df = create_window_size_results(375000)
df

/var/folders/hf/9rstklcn237dv4bv1x1d6x740000gn/T/ipykernel_20897/3988943723.py:18: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, pd.DataFrame([reference_dict])], ignore_index=True)
/var/folders/hf/9rstklcn237dv4bv1x1d6x740000gn/T/ipykernel_20897/3988943723.py:36: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  id_data = pd.concat([id_data, pd.DataFrame([id_data_dict])], ignore_index=True)


,set,total_snps_found,total_snps_overlap,total_loci_overlap,total_coding_snps,average_p_value
0,reference,57.0000,7.0000,34.00,7.00,3.616732e-07
1,random_aggregated_mean,52.4375,8.1875,35.25,6.25,3.082382e-07
2,random_aggregated_median,52.0000,8.0000,35.00,7.00,3.076975e-07
3,random_aggregated_max,55.0000,10.0000,37.00,8.00,3.627492e-07
4,random_aggregated_min,47.0000,6.0000,33.00,3.00,2.711803e-07


Similar results to above, the reference value is n = 37 with 0 coding snps. 
### Result for window size = 250000


In [8]:
df = create_window_size_results(250000)
df

/var/folders/hf/9rstklcn237dv4bv1x1d6x740000gn/T/ipykernel_20897/3988943723.py:18: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, pd.DataFrame([reference_dict])], ignore_index=True)
/var/folders/hf/9rstklcn237dv4bv1x1d6x740000gn/T/ipykernel_20897/3988943723.py:36: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  id_data = pd.concat([id_data, pd.DataFrame([id_data_dict])], ignore_index=True)


,set,total_snps_found,total_snps_overlap,total_loci_overlap,total_coding_snps,average_p_value
0,reference,61.0000,7.0000,41.00,8.000,3.616732e-07
1,random_aggregated_mean,55.6875,8.1875,42.25,7.375,3.082382e-07
2,random_aggregated_median,56.0000,8.0000,42.00,8.000,3.076975e-07
3,random_aggregated_max,59.0000,10.0000,44.00,10.000,3.627492e-07
4,random_aggregated_min,49.0000,6.0000,40.00,4.000,2.711803e-07


Similar results to above, the reference value is n = 44. 

Initial impressions: This analysis was not what I expected. I will perform some additional analyses, however it seems that we may want to reconsider if the tracks should be highly tissue dependent. At 1.0 SD, we see a minor advantage for the brain tracks, so it may be that using these is still beneficial. It may also be that this method is highly effective at eliminating non-causal SNPs, regardless of the track choice.